---
title: PA 4.1 - Webscraping
author: Miranda Jeng
format:
  html:
    theme: cosmo
embed-resources: true
---

# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [1]:
import pandas as pd
import requests

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the ULR using the `requests` library.

In [2]:
# YOUR CODE HERE

response = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")

Use Beautiful Soup to parse this string into a tree called `soup`

In [3]:
# YOUR CODE HERE

from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to the element

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center">
```

There are many `<table>` tags on the page.

In [4]:
len(soup.find_all("table"))

10

We can use attributes like `class=` and `style=` to narrow down the list.

In [5]:
tables = soup.find_all("table")
for table in tables:
  print(table.attrs)

{'class': ['sidebar', 'nomobile', 'nowraplinks']}
{'class': ['wikitable']}
{'class': ['sortable', 'wikitable', 'sticky-header-multi', 'static-row-numbers', 'sort-under', 'col1left', 'col2center'], 'style': 'text-align:right'}
{'class': ['wikitable']}
{'class': ['wikitable'], 'style': 'text-align:left'}
{'class': ['sortable', 'wikitable', 'static-row-numbers', 'sort-under', 'col1left'], 'style': 'text-align:right'}
{'class': ['wikitable', 'sortable', 'sort-under']}
{'class': ['wikitable', 'sortable', 'sort-under', 'col1left', 'col2center'], 'style': 'text-align:right'}
{'class': ['nowraplinks', 'mw-collapsible', 'mw-collapsed', 'navbox-inner'], 'style': 'border-spacing:0;background:transparent;color:inherit'}
{'class': ['navbox-columns-table'], 'style': 'border-spacing: 0px; text-align:left;width:auto; margin-left:auto; margin-right:auto;'}
{'class': ['sidebar', 'nomobile', 'nowraplinks']}
{'class': ['wikitable']}
{'class': ['sortable', 'wikitable', 'sticky-header-multi', 'static-row-nu

In [6]:
len(soup.find_all("table",
                  attrs={
                      "class": ['sortable', 'wikitable', 'sticky-header-multi', 'static-row-numbers', 'sort-under', 'col1left', 'col2center', 'jquery-tablesorter'],
                      "style": "text-align:right"}
                  ))

3

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [7]:
len(soup.find_all("table",
                  attrs={
                      "class": "sticky-header-multi",
                      "style": "text-align:right"}
                  ))

1

In [ ]:
table = soup.find_all("table",
                  attrs={
                      "class": "sticky-header-multi",
                      "style": "text-align:right"}
                  )
print(table[0].prettify())

**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [10]:
# initialize an empty list
rows = []

# Iterate over all rows in the matched table, skipping the header row
for row in table[0].find_all("tr")[1:]:

    # Get all the cells (<td>) in the row
    cells = row.find_all("td")

    # The information we need is the text between tags.

    if len(cells) >= 6:  # Ensure there are enough columns
     city = cells[0].text.strip()
     state = cells[1].text.strip()
     population = cells[2].text.strip()
     land_area_sq_mi = cells[5].text.strip()

     rows.append({
       "city": city,
       "state": state,
       "population (2023 estimate)": population,
       "2020 land area (sq mi)": land_area_sq_mi})

df = pd.DataFrame(rows)
print(df)

            city state population (2023 estimate) 2020 land area (sq mi)
0    New York[c]    NY                  8,258,035                  300.5
1    Los Angeles    CA                  3,820,914                  469.5
2        Chicago    IL                  2,664,452                  227.7
3        Houston    TX                  2,314,157                  640.4
4        Phoenix    AZ                  1,650,070                  518.0
..           ...   ...                        ...                    ...
331         Yuma    AZ                    100,858                  120.7
332  New Bedford    MA                    100,695                   20.0
333   Suffolk[l]    VA                    100,659                  399.2
334     Hesperia    CA                    100,633                   72.7
335    Davenport    IA                    100,354                   63.8

[336 rows x 4 columns]
            city state population (2023 estimate) 2020 land area (sq mi)
0    New York[c]    NY     

Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Aside: Scraping an HTML table with Pandas



The Pandas command `read_html` can be used to scrape information from an HTML table on a webpage.

We can call `read_html` on the URL.

In [11]:
pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")

[                    Population tables of U.S. cities
 0  The skyline of New York City, the most populou...
 1                                             Cities
 2  Population AreaDensityEthnic identityForeign-b...
 3                                        Urban areas
 4             Populous cities and metropolitan areas
 5                                 Metropolitan areas
 6  184 combined statistical areas935 core-based s...
 7                                        Megaregions
 8  See related population listsNorth American met...
 9                                                vte,
     0                               1
 0 NaN                   State capital
 1 NaN              State largest city
 2 NaN  State capital and largest city
 3 NaN                 Federal capital,
             City  ST 2023 estimate  ... 2020 density                                      Location
             City  ST 2023 estimate  ...        / mi2  / km2                               Location
 0    New

However, this scrapes all the tables on the webpage, not just the one we want. As with Beautiful Soup, we can narrow the search by specifying the table attributes.

In [12]:
pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class': 'sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center', "style": "text-align:right"})

[            City  ST 2023 estimate  ... 2020 density                                      Location
             City  ST 2023 estimate  ...        / mi2  / km2                               Location
 0    New York[c]  NY       8258035  ...        29298  11312    40°40′N 73°56′W﻿ / ﻿40.66°N 73.94°W
 1    Los Angeles  CA       3820914  ...         8304   3206  34°01′N 118°25′W﻿ / ﻿34.02°N 118.41°W
 2        Chicago  IL       2664452  ...        12061   4657    41°50′N 87°41′W﻿ / ﻿41.84°N 87.68°W
 3        Houston  TX       2314157  ...         3599   1390    29°47′N 95°23′W﻿ / ﻿29.79°N 95.39°W
 4        Phoenix  AZ       1650070  ...         3105   1199  33°34′N 112°05′W﻿ / ﻿33.57°N 112.09°W
 ..           ...  ..           ...  ...          ...    ...                                    ...
 331         Yuma  AZ        100858  ...          792    306  32°31′N 114°31′W﻿ / ﻿32.52°N 114.52°W
 332  New Bedford  MA        100695  ...         5054   1951    41°40′N 70°56′W﻿ / ﻿41.66°N 70.94°W


This still returns 3 tables. As we remarked above, the table that we want is the first one (see `[0]` below).

In [13]:
df_cities2 = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", attrs = {'class':  'sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center', "style": "text-align:right"})[0]
df_cities2

City  ST 2023 estimate  ... 2020 density                                      Location
            City  ST 2023 estimate  ...        / mi2  / km2                               Location
0    New York[c]  NY       8258035  ...        29298  11312    40°40′N 73°56′W﻿ / ﻿40.66°N 73.94°W
1    Los Angeles  CA       3820914  ...         8304   3206  34°01′N 118°25′W﻿ / ﻿34.02°N 118.41°W
2        Chicago  IL       2664452  ...        12061   4657    41°50′N 87°41′W﻿ / ﻿41.84°N 87.68°W
3        Houston  TX       2314157  ...         3599   1390    29°47′N 95°23′W﻿ / ﻿29.79°N 95.39°W
4        Phoenix  AZ       1650070  ...         3105   1199  33°34′N 112°05′W﻿ / ﻿33.57°N 112.09°W
..           ...  ..           ...  ...          ...    ...                                    ...
331         Yuma  AZ        100858  ...          792    306  32°31′N 114°31′W﻿ / ﻿32.52°N 114.52°W
332  New Bedford  MA        100695  ...         5054   1951    41°40′N 70°56′W﻿ / ﻿41.66°N 70.94°W
333   Suffolk[l]  VA        100659  ...          236     91    36°42′N 76°38′W﻿ / ﻿36.70°N 76.63°W
334     Hesperia  CA        100633  ...         1373    530  34°24′N 117°19′W﻿ / ﻿34.40°N 117.32°W
335    Davenport  IA        100354  ...         1594    615    41°34′N 90°36′W﻿ / ﻿41.56°N 90.60°W

[336 rows x 10 columns]

Wait, that seemed much easier than using Beautiful Soup, and it returned a data frame, and we even got for free some formatting like removing the commas from the population! Why didn't we just use `read_html` in the first place? It's true the `read_html` works well when scraping information from an HTML *table*. Unfortunately, you often want to scrape information from a webpage that isn't conveniently stored in an HTML table, in which case `read_html` won't work. (It only searches for `<table>`, `<th>`, `<tr>`, and `<td>` tags, but there are many other HTML tags.) Though Beautiful Soup is not as simple as `read_html`, it is more flexible and thus more widely applicable.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [14]:
pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")

[                                        Program name   Program type
 0                              Actuarial Preparation          Minor
 1  Cross Disciplinary Studies Minor in Bioinforma...          Minor
 2   Cross Disciplinary Studies Minor in Data Science          Minor
 3                                         Statistics  BS, MS, Minor]

In [15]:
response2 = requests.get("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")
from bs4 import BeautifulSoup
soup2 = BeautifulSoup(response2.content, "html.parser")


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [16]:
rows = []

# Find the relevant section containing course information
course_blocks = soup2.find_all("div", {"class": "courseblock"})

# Iterate over all course entries
for course in course_blocks:
        # Extract course name from the <p> tag with class 'courseblocktitle'
    course_name_tag = course.find("p", {"class": "courseblocktitle"})
    course_name = course_name_tag.text.strip()

        # Extract term typically offered from the first <p> tag with class 'noindent'
    term_offered_tag = course.find("p", {"class": "noindent"})
    term_offered = term_offered_tag.text.strip()

        # Append the data to the list
    rows.append({
        "course name and number": course_name,
        "term typically offered": term_offered
        })

# Create a Pandas DataFrame from the collected data
df_courses = pd.DataFrame(rows)

print(df_courses)

                               course name and number            term typically offered
0          DATA 100. Data Science for All I.\n4 units  Term Typically Offered: F, W, SP
1    DATA 301. Introduction to Data Science.\n4 units  Term Typically Offered: F, W, SP
2   DATA 401. Data Science Process and Ethics.\n3 ...         Term Typically Offered: F
3   DATA 402. Mathematical Foundations of Data Sci...         Term Typically Offered: F
4   DATA 403. Data Science Projects Laboratory.\n1...         Term Typically Offered: F
..                                                ...                               ...
69    STAT 551. Statistical Learning with R.\n4 units         Term Typically Offered: F
70  STAT 566. Graduate Consulting Practicum.\n2 units        Term Typically Offered: SP
71     STAT 570. Selected Advanced Topics.\n1-4 units       Term Typically Offered: TBD
72  STAT 590. Graduate Seminar in Statistics.\n1 unit  Term Typically Offered: F, W, SP
73                       STAT 59

Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag